In [1]:
import pandas as pd
import numpy as np

In [ ]:
# 1.Main Crop Production Dataset
df_crop = pd.read_csv("../data/raw_data/crop-wise-area-production-yield.csv")

In [5]:
df_crop.head()

,id,year,state_name,state_code,district_name,district_code,season,crop_code,crop_name,crop_type,area,area_unit,production,production_unit,yield,yield_unit
0,0,1998-99,Telangana,28,Adilabad,501,Kharif,202,Arhar/Tur,Pulses,35900.0,Hectare,12700.0,Tonnes,0.35,Tonnes/Hectare
1,1,1998-99,Telangana,28,Adilabad,501,Rabi,202,Arhar/Tur,Pulses,100.0,Hectare,100.0,Tonnes,1.00,Tonnes/Hectare
2,2,1998-99,Andhra Pradesh,28,Anantapur,502,Kharif,202,Arhar/Tur,Pulses,27400.0,Hectare,9100.0,Tonnes,0.33,Tonnes/Hectare
3,3,1998-99,Andhra Pradesh,28,Chittoor,503,Kharif,202,Arhar/Tur,Pulses,6100.0,Hectare,1000.0,Tonnes,0.16,Tonnes/Hectare
4,4,1998-99,Andhra Pradesh,28,East Godavari,505,Kharif,202,Arhar/Tur,Pulses,2100.0,Hectare,300.0,Tonnes,0.14,Tonnes/Hectare


In [ ]:
# --- I'll check for missing values ---
df_crop.isnull().sum()

id                 0
year               0
state_name         0
state_code         0
district_name      0
district_code      0
season             0
crop_code          0
crop_name          0
crop_type          0
area               0
area_unit          0
production         0
production_unit    0
yield              0
yield_unit         0
dtype: int64

In [ ]:
# 2. Agronomic Dataset (Rainfall, Fertilizer, Pesticide)
df_agronomic = pd.read_csv("../data/raw_data/crop_yield.csv")

In [20]:
df_agronomic.head()

,Crop,Crop_Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,2051.4,7024878.38,22882.34,0.796087
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,2051.4,631643.29,2057.47,0.710435
2,Castor seed,1997,Kharif,Assam,796.0,22,2051.4,75755.32,246.76,0.238333
3,Coconut,1997,Whole Year,Assam,19656.0,126905000,2051.4,1870661.52,6093.36,5238.051739
4,Cotton(lint),1997,Kharif,Assam,1739.0,794,2051.4,165500.63,539.09,0.420909


In [21]:
# --- I'll check for missing values ---
df_agronomic.isnull().sum()

Crop               0
Crop_Year          0
Season             0
State              0
Area               0
Production         0
Annual_Rainfall    0
Fertilizer         0
Pesticide          0
Yield              0
dtype: int64

In [ ]:
# 3. Soil_Dataset
soil_data_df= pd.read_csv("../data/raw_data/soil_data.csv") 

In [24]:
soil_data_df.head()

,District,pH
0,Andaman Islands,55.420561
1,Nicobar Islands,52.750000
2,Adilabad,69.975706
3,Anantapur,70.379444
4,Chittoor,68.944206


In [23]:
# --- I'll check for missing values ---
soil_data_df.isnull().sum()

District    0
pH          1
dtype: int64

In [28]:
# I'll fill the single missing pH value with the mean of the entire column.
soil_data_df['pH'].fillna(soil_data_df['pH'].mean(), inplace=True)

C:\Users\Punith\AppData\Local\Temp\ipykernel_11112\3917582625.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  soil_data_df['pH'].fillna(soil_data_df['pH'].mean(), inplace=True)


In [29]:
# --- I'll check for missing values ---
soil_data_df.isnull().sum()

District    0
pH          0
dtype: int64

### Merging the dataset

In [73]:
# 1.Merge Crop Data with Agronomic Dataset (Rainfall, Fertilizer, Pesticide)

In [60]:
# Prepare the Crop Data for Merging
# I'll first use .str.strip() to remove any hidden whitespace,
# then slice the first 4 characters and convert to a number.
df_crop['Year'] = df_crop['year'].str.strip().str.slice(0, 4).astype(int)

In [ ]:
df_crop["Year"].unique()

array([1998, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       1999, 2018, 2017, 2019, 2020, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 1997])

In [61]:
df_crop.columns

Index(['id', 'year', 'state_name', 'state_code', 'district_name',
       'district_code', 'season', 'crop_code', 'crop_name', 'crop_type',
       'area', 'area_unit', 'production', 'production_unit', 'yield',
       'yield_unit', 'Year'],
      dtype='object')

In [62]:
df_crop['area_unit'].unique()

array(['Hectare'], dtype=object)

In [63]:
df_crop['production_unit'].unique()

array(['Tonnes'], dtype=object)

In [64]:
df_crop['yield_unit'].unique()

array(['Tonnes/Hectare'], dtype=object)

In [65]:
# Now I'll select and rename the columns I need, adding the units to the names.
df_crop_clean = df_crop[['state_name', 'district_name',
                         'Year', 'season', 'crop_name', 'area', 'yield']].copy()
df_crop_clean.rename(columns={
    'state_name': 'State',
    'district_name': 'District',
    'season': 'Season',
    'crop_name': 'Crop',
    'area': 'Area_Hectare',
    'yield': 'Yield_Tonnes/Hectare'
}, inplace=True)

In [66]:
df_crop_clean.head()

,State,District,Year,Season,Crop,Area_Hectare,Yield_Tonnes/Hectare
0,Telangana,Adilabad,1998,Kharif,Arhar/Tur,35900.0,0.35
1,Telangana,Adilabad,1998,Rabi,Arhar/Tur,100.0,1.00
2,Andhra Pradesh,Anantapur,1998,Kharif,Arhar/Tur,27400.0,0.33
3,Andhra Pradesh,Chittoor,1998,Kharif,Arhar/Tur,6100.0,0.16
4,Andhra Pradesh,East Godavari,1998,Kharif,Arhar/Tur,2100.0,0.14


In [68]:
df_crop_clean['Crop'].unique()

array(['Arhar/Tur', 'Bajra', 'Castor Seed', 'Cotton(Lint)',
       'Dry Chillies', 'Gram', 'Groundnut', 'Horse-Gram', 'Jowar',
       'Maize', 'Mesta', 'Moong(Green Gram)', 'Other  Rabi Pulses',
       'Other Kharif Pulses', 'Ragi', 'Rice', 'Sesamum', 'Small Millets',
       'Sugarcane', 'Sunflower', 'Tobacco', 'Urad', 'Wheat', 'Ginger',
       'Potato', 'Rapeseed &Mustard', 'Soyabean', 'Turmeric', 'Arecanut',
       'Banana', 'Coconut', 'Jute', 'Linseed', 'Niger Seed', 'Onion',
       'Sweet Potato', 'Tapioca', 'Barley', 'Coriander', 'Garlic',
       'Safflower', 'Sannhamp', 'Masoor', 'Moth', 'Other Cereals',
       'Peas & Beans (Pulses)', 'Black Pepper', 'Cardamom', 'Cashewnut',
       'Other Oilseeds', 'Khesari', 'Guar Seed', 'Cowpea(Lobia)',
       'Other Summer Pulses'], dtype=object)

In [101]:
# --- 1. I'll prepare the agronomic data for the merge ---
# I'll select only the columns we need to avoid duplicates.
agronomic_features = df_agronomic[[
    'State', 'Crop_Year', 'Crop', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']].copy()

In [102]:
# I'll rename the columns for consistency before merging.
agronomic_features.rename(columns={
    'Crop_Year': 'Year',
    'Annual_Rainfall': 'Rainfall_mm',
    'Fertilizer': 'Fertilizer_kg',  
    'Pesticide': 'Pesticide_kg'
}, inplace=True)

In [103]:
# I'll standardize the 'State' and 'Crop' columns to title case to match the crop data.
agronomic_features['State'] = agronomic_features['State'].str.title()
agronomic_features['Crop'] = agronomic_features['Crop'].str.title()

In [93]:
df_crop_clean.shape

(345273, 7)

In [94]:
df_agronomic.shape

(19689, 10)

In [99]:
print(df_crop_clean['State'].unique())
print(df_crop_clean['Year'].unique())
print(df_crop_clean['Crop'].unique())

['Telangana' 'Andhra Pradesh' 'Arunachal Pradesh' 'Assam' 'Bihar'
 'Jharkhand' 'Chandigarh' 'The Dadra And Nagar Haveli And Daman And Diu'
 'Delhi' 'Goa' 'Gujarat' 'Haryana' 'Himachal Pradesh' 'Karnataka' 'Kerala'
 'Madhya Pradesh' 'Chhattisgarh' 'Maharashtra' 'Manipur' 'Meghalaya'
 'Mizoram' 'Nagaland' 'Odisha' 'Puducherry' 'Punjab' 'Rajasthan' 'Sikkim'
 'Tamil Nadu' 'Tripura' 'Uttar Pradesh' 'Uttarakhand' 'West Bengal'
 'Andaman And Nicobar Islands' 'Jammu And Kashmir' 'Ladakh']
[1998 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 1999 2018 2017
 2019 2020 2000 2001 2002 2003 2004 2005 2006 1997]
['Arhar/Tur' 'Bajra' 'Castor Seed' 'Cotton(Lint)' 'Dry Chillies' 'Gram'
 'Groundnut' 'Horse-Gram' 'Jowar' 'Maize' 'Mesta' 'Moong(Green Gram)'
 'Other  Rabi Pulses' 'Other Kharif Pulses' 'Ragi' 'Rice' 'Sesamum'
 'Small Millets' 'Sugarcane' 'Sunflower' 'Tobacco' 'Urad' 'Wheat' 'Ginger'
 'Potato' 'Rapeseed &Mustard' 'Soyabean' 'Turmeric' 'Arecanut' 'Banana'
 'Coconut' 'Jute' 'Linseed' 'Niger

In [100]:
print(agronomic_features['State'].unique())
print(agronomic_features['Year'].unique())
print(agronomic_features['Crop'].unique())

['Assam' 'Karnataka' 'Kerala' 'Meghalaya' 'West Bengal' 'Puducherry' 'Goa'
 'Andhra Pradesh' 'Tamil Nadu' 'Odisha' 'Bihar' 'Gujarat' 'Madhya Pradesh'
 'Maharashtra' 'Mizoram' 'Punjab' 'Uttar Pradesh' 'Haryana'
 'Himachal Pradesh' 'Tripura' 'Nagaland' 'Chhattisgarh' 'Uttarakhand'
 'Jharkhand' 'Delhi' 'Manipur' 'Jammu And Kashmir' 'Telangana'
 'Arunachal Pradesh' 'Sikkim']
[1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010
 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020]
['Arecanut' 'Arhar/Tur' 'Castor seed' 'Coconut ' 'Cotton(lint)'
 'Dry chillies' 'Gram' 'Jute' 'Linseed' 'Maize' 'Mesta' 'Niger seed'
 'Onion' 'Other  Rabi pulses' 'Potato' 'Rapeseed &Mustard' 'Rice'
 'Sesamum' 'Small millets' 'Sugarcane' 'Sweet potato' 'Tapioca' 'Tobacco'
 'Turmeric' 'Wheat' 'Bajra' 'Black pepper' 'Cardamom' 'Coriander' 'Garlic'
 'Ginger' 'Groundnut' 'Horse-gram' 'Jowar' 'Ragi' 'Cashewnut' 'Banana'
 'Soyabean' 'Barley' 'Khesari' 'Masoor' 'Moong(Green Gram)'
 'Other Kharif pulses' '

In [104]:
# --- 2. I'll aggregate the data to create unique keys ---
# I'm grouping by the key and taking the mean.
agronomic_aggregated = agronomic_features.groupby(
    ['State', 'Year', 'Crop']).mean().reset_index()

In [105]:
# --- 3. Now, I'll merge with the aggregated data ---
df_merged_agronomic = pd.merge(
    df_crop_clean,
    agronomic_aggregated,
    on=['State', 'Year', 'Crop'],
    how='left'
)

In [106]:
df_merged_agronomic.shape

(345273, 10)

In [108]:
df_merged_agronomic.head(20)

,State,District,Year,Season,Crop,Area_Hectare,Yield_Tonnes/Hectare,Rainfall_mm,Fertilizer_kg,Pesticide_kg
0,Telangana,Adilabad,1998,Kharif,Arhar/Tur,35900.0,0.35,NaN,NaN,NaN
1,Telangana,Adilabad,1998,Rabi,Arhar/Tur,100.0,1.00,NaN,NaN,NaN
2,Andhra Pradesh,Anantapur,1998,Kharif,Arhar/Tur,27400.0,0.33,1048.3,19458660.0,57115.5
3,Andhra Pradesh,Chittoor,1998,Kharif,Arhar/Tur,6100.0,0.16,1048.3,19458660.0,57115.5
4,Andhra Pradesh,East Godavari,1998,Kharif,Arhar/Tur,2100.0,0.14,1048.3,19458660.0,57115.5
5,Andhra Pradesh,Guntur,1998,Kharif,Arhar/Tur,27100.0,0.76,1048.3,19458660.0,57115.5
6,Andhra Pradesh,Guntur,1998,Rabi,Arhar/Tur,200.0,1.00,1048.3,19458660.0,57115.5
7,Andhra Pradesh,Kadapa,1998,Kharif,Arhar/Tur,7600.0,0.58,1048.3,19458660.0,57115.5
8,Andhra Pradesh,Karimnagar,1998,Kharif,Arhar/Tur,5400.0,0.37,1048.3,19458660.0,57115.5
9,Andhra Pradesh,Karimnagar,1998,Rabi,Arhar/Tur,300.0,0.33,1048.3,19458660.0,57115.5


In [109]:
df_merged_agronomic.isnull().sum()

State                       0
District                    0
Year                        0
Season                      0
Crop                        0
Area_Hectare                0
Yield_Tonnes/Hectare        0
Rainfall_mm             27795
Fertilizer_kg           27795
Pesticide_kg            27795
dtype: int64

In [110]:
# I'll define the columns that have missing values.
cols_to_fill = ['Rainfall_mm', 'Fertilizer_kg', 'Pesticide_kg']

In [111]:
# For each column, I'll group by State and Crop, then interpolate.
# This fills missing years based on the trend for that specific crop in that state.
for col in cols_to_fill:
    df_merged_agronomic[col] = df_merged_agronomic.groupby(['State', 'Crop'])[col].transform(
        lambda x: x.interpolate(method='linear')
    )

In [112]:
df_merged_agronomic.isnull().sum()

State                       0
District                    0
Year                        0
Season                      0
Crop                        0
Area_Hectare                0
Yield_Tonnes/Hectare        0
Rainfall_mm             26684
Fertilizer_kg           26684
Pesticide_kg            26684
dtype: int64

In [113]:
# Interpolation can't fill values at the very beginning or end of a group.
# I'll use back-fill and forward-fill to handle any remaining NaNs.
df_merged_agronomic.fillna(method='bfill', inplace=True)
df_merged_agronomic.fillna(method='ffill', inplace=True)

C:\Users\Punith\AppData\Local\Temp\ipykernel_11112\43520398.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged_agronomic.fillna(method='bfill', inplace=True)
C:\Users\Punith\AppData\Local\Temp\ipykernel_11112\43520398.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged_agronomic.fillna(method='ffill', inplace=True)


In [114]:
df_merged_agronomic.isnull().sum()

State                   0
District                0
Year                    0
Season                  0
Crop                    0
Area_Hectare            0
Yield_Tonnes/Hectare    0
Rainfall_mm             0
Fertilizer_kg           0
Pesticide_kg            0
dtype: int64

In [115]:
#3. mrege with soil data
# I'll standardize the 'District' column names to ensure a clean merge.
df_merged_agronomic['District'] = df_merged_agronomic['District'].str.title()
soil_data_df['District'] = soil_data_df['District'].str.title()

In [120]:
soil_data_df['District'].unique()

array(['Andaman Islands', 'Nicobar Islands', 'Adilabad', 'Anantapur',
       'Chittoor', 'Cuddapah', 'East Godavari', 'Guntur', 'Hyderabad',
       'Karimnagar', 'Khammam', 'Krishna', 'Kurnool', 'Mahbubnagar',
       'Medak', 'Nalgonda', 'Nellore', 'Nizamabad', 'Prakasam',
       'Rangareddi', 'Srikakulam', 'Vishakhapatnam', 'Vizianagaram',
       'Warangal', 'West Godavari', 'Changlang', 'East Kameng',
       'East Siang', 'Kurung Kumey', 'Lohit', 'Lower Dibang Valley',
       'Lower Subansiri', 'Papum Pare', 'Tawang', 'Tirap',
       'Upper Dibang Valley', 'Upper Siang', 'Upper Subansiri',
       'West Kameng', 'West Siang', 'Barpeta', 'Bongaigaon', 'Cachar',
       'Darrang', 'Dhemaji', 'Dhuburi', 'Dibrugarh', 'Goalpara',
       'Golaghat', 'Hailakandi', 'Jorhat', 'Kamrup', 'Karbi Anglong',
       'Karimganj', 'Kokrajhar', 'Lakhimpur', 'Marigaon', 'Nagaon',
       'Nalbari', 'North Cachar Hills', 'Sibsagar', 'Sonitpur',
       'Tinsukia', 'Araria', 'Aurangabad', 'Banka', 'Begusarai'

In [ ]:
# ---  I'll remove duplicates from the soil data ---
# I'm grouping by 'District' and taking the mean to ensure each district is unique.
soil_data_unique = soil_data_df.groupby('District').mean().reset_index()

In [122]:
# I'm using a 'left' merge to keep all the rows from our main dataset.
df_final = pd.merge(
    df_merged_agronomic,
    soil_data_unique,  
    on='District',
    how='left'
)

In [123]:
df_final.shape

(345273, 11)

In [124]:
df_final.isnull().sum()

State                       0
District                    0
Year                        0
Season                      0
Crop                        0
Area_Hectare                0
Yield_Tonnes/Hectare        0
Rainfall_mm                 0
Fertilizer_kg               0
Pesticide_kg                0
pH                      76912
dtype: int64

In [125]:
# I'll fill the missing pH values with the mean pH of their respective State.
df_final['pH'] = df_final.groupby(
    'State')['pH'].transform(lambda x: x.fillna(x.mean()))

In [126]:
df_final.isnull().sum()

State                      0
District                   0
Year                       0
Season                     0
Crop                       0
Area_Hectare               0
Yield_Tonnes/Hectare       0
Rainfall_mm                0
Fertilizer_kg              0
Pesticide_kg               0
pH                      1601
dtype: int64

In [127]:
# It's possible a few NaNs remain if an entire state had no matching soil data.
# I'll fill these last few gaps with the overall mean pH from the entire dataset.
df_final['pH'].fillna(df_final['pH'].mean(), inplace=True)

C:\Users\Punith\AppData\Local\Temp\ipykernel_11112\2475029973.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['pH'].fillna(df_final['pH'].mean(), inplace=True)


In [128]:
df_final.isnull().sum()

State                   0
District                0
Year                    0
Season                  0
Crop                    0
Area_Hectare            0
Yield_Tonnes/Hectare    0
Rainfall_mm             0
Fertilizer_kg           0
Pesticide_kg            0
pH                      0
dtype: int64

In [130]:
df_final.to_csv("../data/final_cleaned_crop_data.csv", index=False)